# Linear Optimization - Shipping

Shipping example

In [11]:
# Reference:  https://coin-or.github.io/pulp/

# Dependencies 
from pulp import *

In [12]:
# Parameters
customers=['Australia','Sweeden','Brazil']
factory= ['Factory1','Factory2']
products= ['Chair','Table','Beds']


In [13]:
# create cost table equivalent - this list aligns with the keys

# create a list of all combinations
keys= [(f,p,c) for f in factory for p in products for c in customers]

# (factory1: chair x 3 countries, table x 3, bed x 3), (factory2: chair x 3, table x 3, bed x 3)
costs_value= [50,80,50,
              60,90,60,
              70,90,70,
              80,50,80,
              90,60,90,
              90,70,90]

# pair the keys with cost values and convert to dictionary
costs= dict(zip(keys,costs_value))

In [15]:
# Create linear programming model
model= LpProblem('shipping',LpMinimize)

# add model variable
var= LpVariable.dicts(name='shipment', indices=keys, lowBound=0, upBound=None, cat='Integer')

# name – The prefix to the name of each LP variable created
# indices – A list of strings of the keys to the dictionary of LP variables, and the main part of the variable name itself
# lowBound – The lower bound on these variables’ range. Default is negative infinity
# upBound – The upper bound on these variables’ range. Default is positive infinity
# cat – The category these variables are in, Integer or Continuous(default)
# indexs – (deprecated) Replaced with indices parameter

In [5]:
# create demand table equivalent

# create list of all combinations
demand_keys= [(p,c)for c in customers
              for p in products]

# (Australia: chair, table, bed), (Sweden: ...),(Brazil: ...)
demand_values=[50,80,200,
               120,80,40,
               30,60,175]  

# pair the keys with the demand values and convert to dictionary
demand= dict(zip(demand_keys,demand_values))

In [6]:
# create model constraint that sums the product of cost and product sent (aka the varying variable :: var)
model+= lpSum(var[(f,p,c)]*costs[(f,p,c)]
   for f in factory for p in products for c in customers )

# create model constraint where the sum of the product sent is less than 500 (total capacity)
model += lpSum(var[('Factory1',p,c)]
               for p in products for c in customers)<= 500
model += lpSum(var[('Factory2',p,c)]
               for p in products for c in customers)<= 500

In [7]:
# create model relationship where the product sent (var) for each product is greater than the demand
# goal is to supply all customers but remain under factory capacity
for c in customers:
    for p in products:
        model += var[('Factory1',p,c)]+var[('Factory2',p,c)]>= demand[(p,c)]


In [8]:
# run the model
model.solve()

# print results
for i in var: 
    print('{} shipping {}'.format(i,var[i].varValue))

('Factory1', 'Chair', 'Australia') shipping 50.0
('Factory1', 'Chair', 'Sweeden') shipping 0.0
('Factory1', 'Chair', 'Brazil') shipping 30.0
('Factory1', 'Table', 'Australia') shipping 80.0
('Factory1', 'Table', 'Sweeden') shipping 0.0
('Factory1', 'Table', 'Brazil') shipping 60.0
('Factory1', 'Beds', 'Australia') shipping 105.0
('Factory1', 'Beds', 'Sweeden') shipping 0.0
('Factory1', 'Beds', 'Brazil') shipping 175.0
('Factory2', 'Chair', 'Australia') shipping 0.0
('Factory2', 'Chair', 'Sweeden') shipping 120.0
('Factory2', 'Chair', 'Brazil') shipping 0.0
('Factory2', 'Table', 'Australia') shipping 0.0
('Factory2', 'Table', 'Sweeden') shipping 80.0
('Factory2', 'Table', 'Brazil') shipping 0.0
('Factory2', 'Beds', 'Australia') shipping 95.0
('Factory2', 'Beds', 'Sweeden') shipping 40.0
('Factory2', 'Beds', 'Brazil') shipping 0.0
